In [8]:
# %load data_process.py
import os
import pandas as pd
from collections import defaultdict
import json
from datetime import datetime
from predict_seq import get_result

data_path = '../../assets/初赛数据/'
ouput_path = '../../baseline/mountain_baseline/data/'
from collections import defaultdict  # 导入defaultdict用于创建具有默认值的字典
import pandas as pd  # 导入pandas库用于数据处理
import os  # 导入os库用于文件和目录操作


def merge_csv_files(folder_path, out_path):
    """
    合并CSV文件并导出到指定路径的函数

    主要功能:
    1. 扫描指定文件夹中的所有CSV文件
    2. 将具有相同前缀的文件分组
    3. 合并同组的文件
    4. 将合并结果保存到指定路径

    参数:
    folder_path (str): 源文件夹路径，包含需要合并的CSV文件
    out_path (str): 输出文件夹路径，用于保存合并后的文件

    工作流程:
    1. 首先按文件名前缀对文件进行分组
    2. 对每组文件进行合并操作
    3. 将Excel文件转换为CSV格式
    """

    # 创建一个defaultdict对象用于存储文件分组
    # defaultdict的特点是:当访问不存在的键时,会自动创建一个空列表
    # 这比普通字典更方便,因为不需要先检查键是否存在
    file_groups = defaultdict(list)

    # 遍历源文件夹中的所有文件
    for file_name in os.listdir(folder_path):
        # 检查两个条件:
        # 1. 文件是否以.csv结尾
        # 2. 文件名中是否不包含'字段释义'
        if file_name.endswith('.csv') and '字段释义' not in file_name:
            # rsplit('_', 1)的作用是:
            # 1. 从右侧开始分割文件名
            # 2. 以'_'为分隔符
            # 3. 最多分割1次
            # 例如: 'data_2023_1.csv' -> ['data_2023', '1.csv']
            prefix = file_name.rsplit('_', 1)[0]

            # 使用os.path.join连接路径
            # 这比直接用+连接字符串更安全,因为它会根据操作系统自动使用正确的路径分隔符
            file_groups[prefix].append(os.path.join(folder_path, file_name))

    # 遍历每个分组进行合并操作
    for prefix, file_list in file_groups.items():
        # 使用pandas的concat函数合并同一组的所有CSV文件
        # (pd.read_csv(file) for file in file_list) 是一个生成器表达式
        # 它的优点是:
        # 1. 节省内存 - 不会同时加载所有文件
        # 2. 惰性执行 - 只在需要时才读取文件
        merged_df = pd.concat(
            (pd.read_csv(file) for file in file_list),
            ignore_index=True  # 重置索引,避免合并后的重复索引
        )

        # 构造输出文件路径
        output_file = os.path.join(out_path, f'{prefix}.csv')

        # 创建输出文件所需的目录
        # exist_ok=True表示:如果目录已存在,不会报错
        print('-----------')
        print(output_file)
        os.makedirs(os.path.dirname(output_file), exist_ok=True)

        # 将合并后的数据框保存为CSV文件
        # index=False表示不保存行索引
        merged_df.to_csv(output_file, index=False)

        # 打印进度信息
        print('---完成---')
        print(f'Merged files with prefix "{prefix}" into {output_file}')

    # 处理Excel文件
    # 1. 创建database_in_use目录(如果不存在)
    os.makedirs('database_in_use', exist_ok=True)

    # 2. 读取Excel文件
    # df_device = pd.read_excel(f'{data_path}设备参数详情.xlsx')
    df_device = pd.read_excel(f'{data_path}设备参数详情.xlsx',engine='openpyxl')

    # 3. 将Excel文件保存为CSV格式
    # 保存两份相同的文件到不同位置
    # df_device.to_csv('data/设备参数详情表.csv', index=False)
    # df_device.to_csv('database_in_use/设备参数详情表.csv', index=False)

    df_device.to_csv('data/设备参数详情表.csv', index=False, encoding='utf-8-sig')
    df_device.to_csv('database_in_use/设备参数详情表.csv', index=False, encoding='utf-8-sig')


# 可能的错误情况及处理建议:
# 1. FileNotFoundError: 源文件夹不存在或无法访问
# 2. PermissionError: 没有足够的权限读取或写入文件
# 3. MemoryError: 文件太大,内存不足
# 4. pandas.errors.EmptyDataError: CSV文件为空


In [9]:

# # In[3]:
# # %%
# # Pre2: Status and Events Detection
# # 定义一个函数，将值转换为数值类型，无法转换的返回 -1
# def convert_to_numeric(value):
#     try:
#         return float(value)
#     except ValueError:
#         return -1
# 
# 
# # 读取CSV文件
# df = pd.read_csv('data/Ajia_plc_1.csv')
# # 将 Ajia-3_v 和 Ajia-5_v 列转换为数值类型，无法转换的设为 -1
# df['Ajia-3_v'] = df['Ajia-3_v'].apply(convert_to_numeric)
# df['Ajia-5_v'] = df['Ajia-5_v'].apply(convert_to_numeric)
# # 初始化 status 列，默认值为 'False'
# df['status'] = 'False'
# df['check_current_presence'] = 'False'
# # 遍历每一行，判断设备状态
# for i in range(1, df.shape[0]):
# 
#     # 关机条件
#     if df.loc[i, 'Ajia-5_v'] == -1 and (df.loc[i - 1, 'Ajia-5_v'] > 0 or df.loc[i - 1, 'Ajia-5_v'] == '0'):
#         df.loc[i, 'status'] = '停电'
#         # 开机条件
#     if df.loc[i - 1, 'Ajia-3_v'] == -1 and (df.loc[i, 'Ajia-3_v'] == 0 or df.loc[i, 'Ajia-3_v'] == '0'):
#         df.loc[i, 'status'] = '开机'
#     if df.loc[i - 1, 'Ajia-3_v'] == -1 and df.loc[i, 'Ajia-3_v'] > 0:
#         df.loc[i, 'status'] = '开机'
#     if df.loc[i - 1, 'Ajia-5_v'] == -1 and (df.loc[i, 'Ajia-5_v'] == 0 or df.loc[i, 'Ajia-5_v'] == '0'):
#         df.loc[i, 'status'] = '开机'
#     # 关机条件
#     if df.loc[i, 'Ajia-3_v'] == -1 and (df.loc[i - 1, 'Ajia-3_v'] == 0 or df.loc[i - 1, 'Ajia-3_v'] == '0'):
#         df.loc[i, 'status'] = '关机'
#     if df.loc[i, 'Ajia-5_v'] == -1 and (df.loc[i - 1, 'Ajia-5_v'] == 0 or df.loc[i - 1, 'Ajia-5_v'] == '0'):
#         df.loc[i, 'status'] = '关机'
#     if df.loc[i, 'Ajia-5_v'] > 0 and df.loc[i - 1, 'Ajia-5_v'] == -1:
#         df.loc[i, 'check_current_presence'] = '有电流'
#     if df.loc[i, 'Ajia-5_v'] > 0 and (df.loc[i - 1, 'Ajia-5_v'] == 0 or df.loc[i - 1, 'Ajia-5_v'] == '0'):
#         df.loc[i, 'check_current_presence'] = '有电流'
#     if df.loc[i, 'Ajia-5_v'] == 0 and (df.loc[i - 1, 'Ajia-5_v'] > 0 or df.loc[i - 1, 'Ajia-5_v'] == '0'):
#         df.loc[i, 'check_current_presence'] = '无电流'



def process_ajia_data(csv_path):
    """
    处理A架数据，检测设备状态和电流存在情况
    
    参数:
    csv_path: CSV文件路径
    
    返回:
    处理后的DataFrame
    """
    # 1. 读取和基础处理
    df = pd.read_csv(csv_path)
    
    # 2. 定义转换函数
    def convert_to_numeric(value):
        """将值转换为数值类型，无法转换的返回 -1"""
        try:
            if isinstance(value, str) and value.strip() == '0':
                return 0
            return float(value)
        except (ValueError, TypeError):
            return -1

    # 3. 转换数值列
    columns_to_convert = ['Ajia-3_v', 'Ajia-5_v']
    for col in columns_to_convert:
        df[col] = df[col].apply(convert_to_numeric)

    # 4. 初始化状态列
    df['status'] = 'False'
    df['check_current_presence'] = 'False'
    
    # 5. 处理第一行（索引0）的状态
    # 根据第一行的数值确定初始状态
    if df.loc[0, 'Ajia-5_v'] > 0:
        df.loc[0, 'check_current_presence'] = '有电流'
    elif df.loc[0, 'Ajia-5_v'] == 0:
        df.loc[0, 'check_current_presence'] = '无电流'
    
    if df.loc[0, 'Ajia-3_v'] == -1:
        df.loc[0, 'status'] = '关机'
    else:
        df.loc[0, 'status'] = '开机'

    # 6. 处理剩余行的状态
    for i in range(1, len(df)):
        # 获取当前行和前一行的值，提高代码可读性
        curr_v3 = df.loc[i, 'Ajia-3_v']
        prev_v3 = df.loc[i-1, 'Ajia-3_v']
        curr_v5 = df.loc[i, 'Ajia-5_v']
        prev_v5 = df.loc[i-1, 'Ajia-5_v']
        
        # 6.1 电流状态检测
        if curr_v5 > 0 and (prev_v5 == -1 or prev_v5 == 0):
            df.loc[i, 'check_current_presence'] = '有电流'
        elif curr_v5 == 0 and prev_v5 > 0:
            df.loc[i, 'check_current_presence'] = '无电流'
        else:
            # 保持前一个状态
            df.loc[i, 'check_current_presence'] = df.loc[i-1, 'check_current_presence']

        # 6.2 设备状态检测
        # 开机条件
        if ((prev_v3 == -1 and (curr_v3 == 0 or curr_v3 > 0)) or
            (prev_v5 == -1 and curr_v5 == 0)):
            df.loc[i, 'status'] = '开机'
        # 关机条件
        elif ((curr_v3 == -1 and prev_v3 in [0, '0']) or
              (curr_v5 == -1 and prev_v5 in [0, '0'])):
            df.loc[i, 'status'] = '关机'
        # 停电条件
        elif curr_v5 == -1 and prev_v5 > 0:
            df.loc[i, 'status'] = '停电'
        else:
            # 保持前一个状态
            df.loc[i, 'status'] = df.loc[i-1, 'status']

    # 7. 验证处理结果
    # 检查是否有未处理的状态（None值）
    unprocessed_status = df['status'].isnull().sum()
    unprocessed_current = df['check_current_presence'].isnull().sum()
    
    if unprocessed_status > 0 or unprocessed_current > 0:
        print(f"警告：存在未处理的状态：\n"
              f"status: {unprocessed_status}行\n"
              f"check_current_presence: {unprocessed_current}行")
    
    return df

# 使用方法
try:
    df = process_ajia_data('data/Ajia_plc_1.csv')
    # 保存处理后的数据
    df.to_csv('data/Ajia_plc_1_processed.csv', index=False)
    print("数据处理完成")
except Exception as e:
    print(f"处理过程中发生错误: {e}")

数据处理完成


In [10]:


def is_mostly_fifty(L_):
    """
    判断列表中是否大部分数值接近50
    
    参数:
    L_ (list): 包含数值的列表，通常是电流值
    
    返回:
    int: 1 表示大部分值在50-60之间（待机状态），0 表示其他情况
    """
    # 数据过滤与处理
    filtered_list = [x for x in L_ if x != 0 and x <= 200]
    # 数值标准化
    normalized_list = [50 if 50 <= x <= 60 else x for x in filtered_list]
    # 统计50的数量
    count_50 = normalized_list.count(50)
    # 判断占比并返回结果
    return 1 if count_50 > len(normalized_list) / 2 else 0

    # 初始化变量


# 初始化变量
start_time = None  # 用于记录开机时间点
segments = []      # 用于存储所有完整的开机-关机时间段

# 遍历数据框中的每一行记录
for index, row in df.iterrows():
    # 检测开机事件
    if row['status'] == '开机':
        start_time = row['csvTime']  # 记录开机时间
    # 检测关机事件（必须在有开机记录的情况下）
    elif row['status'] == '关机' and start_time is not None:
        end_time = row['csvTime']  # 记录关机时间
        segments.append((start_time, end_time))  # 保存这个完整周期
        start_time = None  # 重置开机时间，准备记录下一个周期

In [11]:


def extract_daily_power_on_times(df):
    """
    从CSV文件中提取一天内有两次开机的第一次和第二次开机时间。

    参数:
    file_path (str): CSV文件的路径，包含 'csvTime' 和 'status' 列。

    返回:
    first_start_times (list): 一天内有两次开机的第一次开机时间列表。
    second_start_times (list): 一天内有两次开机的第二次开机时间列表。
    """
    # 读取CSV文件
    df = df

    # 将时间列转换为标准datetime格式
    df['csvTime'] = pd.to_datetime(df['csvTime'])
    # datetime类型可以进行更精确的时间运算和比较

    # 提取日期信息用于分组
    df['date'] = df['csvTime'].dt.date
    # 这一步将时间戳拆分，只保留日期部分，例如：
    # '2024-08-24 09:30:00' -> '2024-08-24'

    # 初始化一个字典来存储每天的开机关机时间段
    daily_segments = {}
    # 数据结构示例：
    # {
    #     datetime.date(2024, 8, 24): [
    #         (datetime('2024-08-24 08:00'), datetime('2024-08-24 11:00')),
    #         (datetime('2024-08-24 14:00'), datetime('2024-08-24 17:00'))
    #     ],
    #     datetime.date(2024, 8, 25): [
    #         ...
    #     ]
    # }
    # 遍历每一天的数据
    for date, group in df.groupby('date'):
        segments = []# 存储当天的开关机时段
        start_time = None# 临时存储开机时间

        # 遍历每一天的记录
        for index, row in group.iterrows():
            if row['status'] == '开机':
                start_time = row['csvTime']
            elif row['status'] == '关机' and start_time is not None:
                end_time = row['csvTime']
                segments.append((start_time, end_time))#segment里面存储的是元组，每个元组是一次开关机时间段
                start_time = None

        # 将每天的开机关机时间段存入字典
        daily_segments[date] = segments

    # 统计每天的开机关机次数
    # 示例：{'2024-08-24': 2, '2024-08-25': 1}
    daily_counts = {date: len(segments) for date, segments in daily_segments.items()}

    # 筛选出一天内有两次开机关机的情况
    two_times_days = [date for date, count in daily_counts.items() if count == 2]

    # 初始化两个列表来存储第一次和第二次的开机时间
    first_start_times = []
    second_start_times = []

    # 遍历这些日期，提取第一次和第二次的开机时间
    for date in two_times_days:
        segments = daily_segments[date]
        first_start_times.append(segments[0][0])  # 第一次开机时间
        second_start_times.append(segments[1][0])  # 第二次开机时间

    return first_start_times, second_start_times


def find_peaks(data1):
    """
    在数据序列中寻找并返回符合特定条件的峰值
    
    功能说明：
    1. 对原始数据进行预处理，将50-68范围内的值标准化为50
    2. 识别局部峰值（比相邻点都大的点）
    3. 筛选大于80的峰值
    
    参数：
    data1: List[float/int] - 输入的数值序列
    
    返回：
    Tuple[int, List[float/int]] - (峰值个数, 峰值列表)
    """
    
    # 数据预处理
    data = [50 if 50 <= num <= 68 else num for num in data1]
    # 这一步的详细分解：
# 1. 遍历原始数据列表
# 2. 对每个数值进行条件判断：
#    - 如果数值在[50, 68]范围内 → 转换为50
#    - 如果数值在范围外 → 保持原值
#
# 示例：
# 输入：[45, 55, 70, 60, 90]
# 输出：[45, 50, 70, 50, 90]
    

    # 找到峰值
    peaks = []
    for i in range(1, len(data) - 1):  # 从第二个元素遍历到倒数第二个元素
        if data[i] > data[i - 1] and data[i] > data[i + 1]:  # 判断是否为峰值
            peaks.append(data[i])  # 只记录峰值值
    peaks = [peak for peak in peaks if peak > 80]
    # 返回峰值格式和具体的峰值
    return len(peaks), peaks


def find_first_increasing_value(data):
    """
    找到列表中第一个从稳定值（68以下）开始增加的值。

    参数:
    data (list): 输入的数值列表。

    返回:
    tuple: 第一个大于68的值及其索引。如果未找到，返回 (None, None)。
    """
    # 将介于50到68之间的值替换为50
    processed_data = [50 if 50 <= num <= 68 else num for num in data]

    # 找到第一个大于68的值及其索引
    for index, value in enumerate(processed_data):
        if value > 68 and value < 300:
            return value
    # 如果未找到，返回 (None, None)
    return 50


def find_stable_value(data, peak1, peak2):
    """
    找到两个峰值之间的数据中，回落到稳定值的第一个值。
    假设稳定值在 50 到 60 之间。

    参数:
    data (list): 数据列表
    peak1 (float): 第一个峰值
    peak2 (float): 第二个峰值

    返回:
    float or None: 稳定值，如果未找到则返回 None
    """
    # 找到峰值之间的数据
    try:
        start_index = data.index(peak1)
        end_index = data.index(peak2)
    except ValueError:
        # 如果峰值不在列表中，返回 None
        return None

    between_peaks = data[start_index:end_index + 1]

    # 找到回落到稳定值的第一个值（假设稳定值在 50 到 60 之间）
    for value in between_peaks:
        if 50 <= value <= 60:
            return value

    # 如果未找到稳定值，返回 None
    return None


def find_first_stable_after_peak(data, peak, stable_min=50, stable_max=60):
    """
    从峰值到列表末尾的数据中，找到第一个回落到稳定值的值。

    参数:
    data (list): 数据列表
    peak (float): 峰值
    stable_min (float): 稳定值的最小值
    stable_max (float): 稳定值的最大值

    返回:
    float or None: 稳定值，如果未找到则返回 None
    """
    try:
        # 找到峰值的索引
        start_index = data.index(peak)
    except ValueError:
        # 如果峰值不在列表中，返回 None
        return None

    # 切片获取从峰值到列表末尾的数据
    after_peak = data[start_index:]

    # 找到回落到稳定值的第一个值
    for value in after_peak:
        if stable_min <= value <= stable_max:
            return value

    # 如果未找到稳定值，返回 None
    return None


LLLL = []
import pandas as pd


def extract_events(df, segment):
    
    """
    从指定的时间段内提取和分析电流事件序列
    
    参数解析：
    df (pd.DataFrame): 包含设备状态数据的数据框
        必需列：
        - csvTime: 时间戳列
        - check_current_presence: 电流状态列（'有电流'/'无电流'）
        - Ajia-5_v: 电流值列
    segment (tuple): 时间段区间元组 (start_time, end_time)
    
    返回：
    List[int]: 每对电流事件中检测到的峰值数量列表
    """
    
    start, end = segment
    # start = pd.to_datetime(start)
    # end = pd.to_datetime(end)
    print(f"开机时间: {start}, 关机时间: {end}")
    
    # 提取满足三个条件的事件：
    # 1. 时间在指定区间内
    # 2. 电流状态为"有电流"或"无电流"
    
    
    events = df[
        (df['csvTime'] >= start) & (df['csvTime'] <= end) & (df['check_current_presence'].isin(['有电流', '无电流']))]
    # 打印找到的事件数量
    print(f"事件数量: {events.shape[0]}")
    
    # 初始化一个列表，用于存储每对事件中检测到的峰值数量
    L3 = []
    
    # 检查事件数量是否为偶数
    # 验证事件条件：
    # 1. 至少有2个事件
    # 2. 事件总数为偶数（确保可以配对）
    if events.shape[0] >= 2 and events.shape[0] % 2 == 0:
        # 遍历所有偶数索引的事件对
        # 步长为2遍历事件，处理事件对
        for i in range(0, events.shape[0], 2):
            # 获取事件对
            event_start = events.iloc[i]
            event_end = events.iloc[i + 1]
            
            # 确保第一个事件是“有电流”，第二个事件是“无电流”
            if event_start['check_current_presence'] == '有电流' and event_end['check_current_presence'] == '无电流':
                
                 # 记录事件的具体时间点
                start_event_time = event_start['csvTime']
                end_event_time = event_end['csvTime']

                # 提取两个事件之间的数据
                between_events = df[(df['csvTime'] >= start_event_time) & (df['csvTime'] <= end_event_time)]
                 
                 # 提取电流值序列
                data1 = list(between_events['Ajia-5_v'])
                print(f"事件对 ({i}, {i + 1}) 之间的数据: {data1}")

                # 调用 find_peaks 函数（假设已定义）
                len_peaks, peak_L = find_peaks(data1)
                print(f'峰值为{peak_L}')
                L3.append(len_peaks)
    return L3


L5 = [] # 创建一个空列表用于存储所有段的事件分析结果
# 提取每个区段内的“通电流”和“关电流”事件

# 核心循环：遍历所有时间段进行处理
for segment in segments:
    
    # 对每个时间段执行事件提取
    L4 = extract_events(df, segment=segment)
    # 将当前段的分析结果添加到总结果列表
    L5.append(L4)
    # 解构时间段
    start, end = segment
    
    # 提取当前时间段的所有数据
    events_2 = df[(df['csvTime'] >= start) & (df['csvTime'] <= end)]
    
    
    print('-----------------事件--------------------')
    print(list(events_2['Ajia-5_v']))# 打印电流值序列
    print('-----------------事件--------------------')
    
    # 提取有效电流事件（只包含有电流和无电流的状态）
    events_1 = df[
        (df['csvTime'] >= start) & (df['csvTime'] <= end) & (df['check_current_presence'].isin(['有电流', '无电流']))]
    LLLL.append(events_1.shape[0])# 记录有效事件的数量
    # if start=='2024-08-24 07:55:08':

    # 这部分处理峰值模式为[0, 2]的情况，表示第一个事件对没有峰值，第二个事件对有2个峰值。
    if L4 == [0, 2]:
        # 提取当前时间段内的所有电流事件
        events = df[(df['csvTime'] >= start) & (df['csvTime'] <= end) & (
            df['check_current_presence'].isin(['有电流', '无电流']))]
        
        # 检查事件数量是否为偶数
        if events.shape[0] % 2 == 0:
            
            if events.iloc[0]['check_current_presence'] == '有电流' and events.iloc[1][
                'check_current_presence'] == '无电流':
                
                # 提取第一对事件的时间范围
                start_event_time = events.iloc[0]['csvTime']
                end_event_time = events.iloc[1]['csvTime']
                between_events = df[(df['csvTime'] >= start_event_time) & (df['csvTime'] <= end_event_time)]
                data1 = list(between_events['Ajia-5_v'])
                
                # 分析峰值
                len_peaks, peak_L = find_peaks(data1)
                
                
                # 如果第一个事件对没有峰值，检查第二个事件对
                if len_peaks == 0:
                    if events.iloc[2]['check_current_presence'] == '有电流' and events.iloc[3][
                        'check_current_presence'] == '无电流':
                        
                        # 提取第二对事件的数据
                        start_event_time = events.iloc[2]['csvTime']
                        end_event_time = events.iloc[3]['csvTime']
                        between_events = df[(df['csvTime'] >= start_event_time) & (df['csvTime'] <= end_event_time)]
                        data1 = list(between_events['Ajia-5_v'])
                        print(data1)
                        
                        # 分析第二对事件的峰值
                        len_peaks, peak_L = find_peaks(data1)
                        
                        # 如果找到两个峰值，进行状态标记
                        if len_peaks == 2:
                            
                           # 标记征服者起吊 
                            value_11 = find_first_increasing_value(data1)
                            indices = between_events.index[between_events['Ajia-5_v'] == value_11].tolist()
                            df.loc[indices, 'status'] = '征服者起吊'
                            
                            # 标记缆绳解除和征服者入水
                            value_11 = find_stable_value(data1, peak_L[0], peak_L[1])
                            indices = between_events.index[between_events['Ajia-5_v'] == value_11].tolist()
                            df.loc[indices, 'status'] = '缆绳解除'
                            previous_indices = [idx - 1 for idx in indices if idx > 0]
                            df.loc[previous_indices, 'status'] = '征服者入水'

                            # 找到 between_events 中 Ajia-5_v 等于 target_value 的索引
                            indices = between_events.index[between_events['Ajia-5_v'] == peak_L[1]].tolist()
                            df.loc[indices, 'status'] = 'A架摆回'
    if L4 == [2]:
        
        
        """
    处理只有一对电流事件且有2个峰值的情况
    这种模式通常对应一次完整的下放操作
    典型的下放操作流程：
    第一个峰值 -> 征服者起吊
    两峰之间的稳定值 -> 缆绳解除
    第二个峰值 -> A架摆回
    """
        
        # 1. 提取有效事件
        events = df[(df['csvTime'] >= start) & (df['csvTime'] <= end) & (
            df['check_current_presence'].isin(['有电流', '无电流']))]
        
        # 2. 检查事件数量是否为偶数
        if events.shape[0] % 2 == 0:
            # 3. 验证事件对的有效性
            if events.iloc[0]['check_current_presence'] == '有电流' and events.iloc[1][
                'check_current_presence'] == '无电流':
                
                # 4. 提取第一对事件的时间范围
                start_event_time = events.iloc[0]['csvTime']
                end_event_time = events.iloc[1]['csvTime']
                
                # 5. 提取第一对事件的数据
                between_events = df[(df['csvTime'] >= start_event_time) & (df['csvTime'] <= end_event_time)]
                
                # 6. 获取电流值序列
                data1 = list(between_events['Ajia-5_v'])
                
                # 7. 分析峰值
                len_peaks, peak_L = find_peaks(data1)
                
                # 8. 处理双峰模式
                if len_peaks == 2:
                    
                     # 8.1 再次验证事件对（双重检查）
                    if events.iloc[0]['check_current_presence'] == '有电流' and events.iloc[1][
                        'check_current_presence'] == '无电流':
                        
                        # 8.2 重新提取数据（确保数据准确性）
                        start_event_time = events.iloc[0]['csvTime']
                        end_event_time = events.iloc[1]['csvTime']
                        between_events = df[(df['csvTime'] >= start_event_time) & (df['csvTime'] <= end_event_time)]
                        data1 = list(between_events['Ajia-5_v'])
                        
                        # 8.3 再次分析峰值
                        len_peaks, peak_L = find_peaks(data1)
                        
                        # 8.4 处理确认的双峰模式
                        if len_peaks == 2:
                            # a. 标记征服者起吊
                            value_11 = find_first_increasing_value(data1)
                            indices = between_events.index[between_events['Ajia-5_v'] == value_11].tolist()
                            df.loc[indices, 'status'] = '征服者起吊'
                            
                            
                            # b. 标记缆绳解除和征服者入水
                            value_11 = find_stable_value(data1, peak_L[0], peak_L[1])
                            indices = between_events.index[between_events['Ajia-5_v'] == value_11].tolist()
                            df.loc[indices, 'status'] = '缆绳解除'
                            
                            # c. 标记征服者入水（在缆绳解除之前）
                            previous_indices = [idx - 1 for idx in indices if idx > 0]
                            df.loc[previous_indices, 'status'] = '征服者入水'

                            # 找到 between_events 中 Ajia-5_v 等于 target_value 的索引
                            
                            # d. 标记A架摆回
                            indices = between_events.index[between_events['Ajia-5_v'] == peak_L[1]].tolist()
                            df.loc[indices, 'status'] = 'A架摆回'

    elif L4 == [0, 3]:
        """
    处理特定的峰值模式：第一个事件对没有峰值[0]，第二个事件对有3个峰值[3]
    这种模式通常表示一个复杂的设备操作序列
        典型的操作流程："""
          # 1. 提取满足条件的事件
        events = df[(df['csvTime'] >= start) & (df['csvTime'] <= end) & (
            df['check_current_presence'].isin(['有电流', '无电流']))]
        
         # 验证事件数量为偶数（确保事件可以配对）
        if events.shape[0] % 2 == 0:
            # 验证第一对事件的正确性
            if events.iloc[0]['check_current_presence'] == '有电流' and events.iloc[1][
                'check_current_presence'] == '无电流':
                
                # 提取第一对事件的时间范围
                start_event_time = events.iloc[0]['csvTime']
                end_event_time = events.iloc[1]['csvTime']
                
                # 提取第一对事件的数据
                between_events = df[(df['csvTime'] >= start_event_time) & (df['csvTime'] <= end_event_time)]
                data1 = list(between_events['Ajia-5_v'])

                len_peaks, peak_L = find_peaks(data1)
                
                # 如果第一对事件确实没有峰值
                if len_peaks == 0:
                    # 验证第二对事件
                    if events.iloc[2]['check_current_presence'] == '有电流' and events.iloc[3][
                        'check_current_presence'] == '无电流':
                        # 提取第二对事件的时间范围
                        start_event_time = events.iloc[2]['csvTime']
                        end_event_time = events.iloc[3]['csvTime']
                        # 获取第二对事件的数据

                        between_events = df[(df['csvTime'] >= start_event_time) & (df['csvTime'] <= end_event_time)]
                        data1 = list(between_events['Ajia-5_v'])
                        print(data1)
                        
                        # 分析第二对事件的峰值
                        len_peaks, peak_L = find_peaks(data1)
                        
                        # 处理三峰值模式
                        if len_peaks == 3:
                            
                            # a. 标记征服者起吊（使用第一个上升值）
                            value_11 = find_first_increasing_value(data1)
                            indices = between_events.index[between_events['Ajia-5_v'] == value_11].tolist()
                            df.loc[indices, 'status'] = '征服者起吊'
                             # b. 标记缆绳解除（使用第二、三个峰值之间的稳定值）
                            value_11 = find_stable_value(data1, peak_L[1], peak_L[2])
                            indices = between_events.index[between_events['Ajia-5_v'] == value_11].tolist()
                            df.loc[indices, 'status'] = '缆绳解除'
                            previous_indices = [idx - 1 for idx in indices if idx > 0]
                            # c. 标记征服者入水（在缆绳解除之前）
                            df.loc[previous_indices, 'status'] = '征服者入水'

                            # 找到 between_events 中 Ajia-5_v 等于 target_value 的索引
                            
                            # d. 标记A架摆回（使用最后一个峰值）
                            indices = between_events.index[between_events['Ajia-5_v'] == peak_L[2]].tolist()
                            df.loc[indices, 'status'] = 'A架摆回'
    elif L4 == [0, 1, 3]:
        
        events = df[(df['csvTime'] >= start) & (df['csvTime'] <= end) & (
            df['check_current_presence'].isin(['有电流', '无电流']))]
        
        if events.shape[0] % 2 == 0:
            if events.iloc[0]['check_current_presence'] == '有电流' and events.iloc[1][
                'check_current_presence'] == '无电流':
                start_event_time = events.iloc[0]['csvTime']
                end_event_time = events.iloc[1]['csvTime']
                between_events = df[(df['csvTime'] >= start_event_time) & (df['csvTime'] <= end_event_time)]
                data1 = list(between_events['Ajia-5_v'])

                len_peaks, peak_L = find_peaks(data1)

                if len_peaks == 0:
                    if events.iloc[4]['check_current_presence'] == '有电流' and events.iloc[5][
                        'check_current_presence'] == '无电流':
                        start_event_time = events.iloc[4]['csvTime']
                        end_event_time = events.iloc[5]['csvTime']
                        between_events = df[(df['csvTime'] >= start_event_time) & (df['csvTime'] <= end_event_time)]
                        data1 = list(between_events['Ajia-5_v'])

                        len_peaks, peak_L = find_peaks(data1)
                        if len_peaks == 3:
                            value_11 = find_first_increasing_value(data1)
                            indices = between_events.index[between_events['Ajia-5_v'] == value_11].tolist()
                            df.loc[indices, 'status'] = '征服者起吊'
                            value_11 = find_stable_value(data1, peak_L[1], peak_L[2])
                            indices = between_events.index[between_events['Ajia-5_v'] == value_11].tolist()
                            df.loc[indices, 'status'] = '缆绳解除'
                            previous_indices = [idx - 1 for idx in indices if idx > 0]
                            df.loc[previous_indices, 'status'] = '征服者入水'

                            # 找到 between_events 中 Ajia-5_v 等于 target_value 的索引
                            indices = between_events.index[between_events['Ajia-5_v'] == peak_L[2]].tolist()
                            df.loc[indices, 'status'] = 'A架摆回'
    elif L4 == [1, 2] or L4 == [1, 1]:
        events = df[(df['csvTime'] >= start) & (df['csvTime'] <= end) & (
            df['check_current_presence'].isin(['有电流', '无电流']))]
        if events.shape[0] % 2 == 0:
            if events.iloc[0]['check_current_presence'] == '有电流' and events.iloc[1][
                'check_current_presence'] == '无电流':
                start_event_time = events.iloc[0]['csvTime']
                end_event_time = events.iloc[1]['csvTime']
                between_events = df[(df['csvTime'] >= start_event_time) & (df['csvTime'] <= end_event_time)]
                data1 = list(between_events['Ajia-5_v'])

                len_peaks, peak_L = find_peaks(data1)

                if len_peaks == 1:
                    value_11 = find_first_increasing_value(data1)
                    indices = between_events.index[between_events['Ajia-5_v'] == peak_L[0]].tolist()
                    df.loc[indices, 'status'] = 'A架摆出'
                    if events.iloc[2]['check_current_presence'] == '有电流' and events.iloc[3][
                        'check_current_presence'] == '无电流':
                        start_event_time = events.iloc[2]['csvTime']
                        end_event_time = events.iloc[3]['csvTime']
                        between_events = df[(df['csvTime'] >= start_event_time) & (df['csvTime'] <= end_event_time)]
                        data1 = list(between_events['Ajia-5_v'])

                        len_peaks, peak_L = find_peaks(data1)
                        max_value = max([x for x in data1 if x <= 200])

                        if len_peaks == 2:
                            # 找到 between_events 中 Ajia-5_v 等于 target_value 的索引
                            indices = between_events.index[between_events['Ajia-5_v'] == max_value].tolist()
                            df.loc[indices, 'status'] = '征服者出水'
                            previous_indices = [idx - 1 for idx in indices if idx > 0]
                            df.loc[previous_indices, 'status'] = '缆绳挂妥'

                            value_11 = find_first_stable_after_peak(data1, max_value)
                            indices = between_events.index[between_events['Ajia-5_v'] == value_11].tolist()
                            df.loc[indices, 'status'] = '征服者落座'
                        elif len_peaks == 1:
                            # 找到 between_events 中 Ajia-5_v 等于 target_value 的索引
                            indices = between_events.index[between_events['Ajia-5_v'] == max_value].tolist()
                            df.loc[indices, 'status'] = '征服者出水'
                            previous_indices = [idx - 1 for idx in indices if idx > 0]
                            df.loc[previous_indices, 'status'] = '缆绳挂妥'

                            value_11 = find_first_stable_after_peak(data1, max_value)
                            indices = between_events.index[between_events['Ajia-5_v'] == value_11].tolist()
                            df.loc[indices, 'status'] = '征服者落座'
    else:
    # 这是在处理其他峰值模式不匹配的情况
    # 创建数据副本以避免修改原始数据
        events_2 = events_2.copy()
    # 确保时间列的正确格式
        events_2.loc[:, 'csvTime'] = pd.to_datetime(events_2['csvTime'])
        # 获取第一个值
        first_value = events_2['csvTime'].iloc[0]

        # 定义目标日期
        target_date = datetime(2024, 8, 19)  # 2024年8月19日   大模型预测测试
        is_target_date = (first_value.date() == target_date.date())

        target_date_1 = datetime(2024, 8, 19)  # 2024年8月19日   大模型预测测试
        is_target_date_1 = (first_value.date() == target_date.date())

        # 判断小时是否大于12点 判断时间是否在下午5点之后
        is_hour_greater_than_12 = first_value.hour > 17
    
        # 获取每天的开机时间序列
        first_start_times, second_start_times = extract_daily_power_on_times(df=df)
        
        
        # 特定条件判断：目标日期且在下午5点后
        if is_target_date and is_hour_greater_than_12:  # 全部预测可以去掉is_target_date条件 或者根据问题传入
            events_2['new_column'] = events_2.apply(
                lambda row: row['Ajia-3_v'] if row['Ajia-5_v'] == 0 and row['Ajia-3_v'] > 0 else row['Ajia-5_v'],
                axis=1
            )
            print('----------------LLM预测的列表---------------------')
            print(str(list(events_2['new_column'])))

            try:
                a, b, c = get_result(str(list(events_2['new_column'])), 1)
            except Exception as e:
                print(f"An error occurred: {e}")
                a, b, c = -100, -100, -100
            print('----------------预测的值---------------------')
            print(a, b, c)

            indices = events_2.index[events_2['new_column'] == a].tolist()
            df.loc[indices, 'status'] = 'A架摆出'

            indices = events_2.index[events_2['new_column'] == b].tolist()
            df.loc[indices, 'status'] = '征服者出水'
            previous_indices = [idx - 1 for idx in indices if idx > 0]
            df.loc[previous_indices, 'status'] = '缆绳挂妥'

            indices = events_2.index[events_2['new_column'] == c].tolist()
            df.loc[indices, 'status'] = '征服者落座'



        elif first_value in first_start_times and 1 == 0:  # 去掉1==0由LLM判断状态  默认不开启 给大家分享参考思路。
            events_2['new_column'] = events_2.apply(
                lambda row: row['Ajia-3_v'] if row['Ajia-5_v'] == 0 and row['Ajia-3_v'] > 0 else row['Ajia-5_v'],
                axis=1
            )
            print('----------------LLM预测的列表---------------------')
            print(str(list(events_2['new_column'])))

            try:
                a, b, c = get_result(str(list(events_2['new_column'])), 0)
            except Exception as e:
                print(f"An error occurred: {e}")
                a, b, c = -100, -100, -100
            print('----------------预测的值---------------------')
            print(a, b, c)

            indices = events_2.index[events_2['new_column'] == a].tolist()
            df.loc[indices, 'status'] = '征服者起吊'

            indices = events_2.index[events_2['new_column'] == b].tolist()
            df.loc[indices, 'status'] = '缆绳解除'
            previous_indices = [idx - 1 for idx in indices if idx > 0]
            df.loc[previous_indices, 'status'] = '征服者入水'

            indices = events_2.index[events_2['new_column'] == c].tolist()
            df.loc[indices, 'status'] = 'A架摆回'

        elif first_value in second_start_times and 1 == 0:  # 去掉1==0由LLM判断状态
            events_2['new_column'] = events_2.apply(
                lambda row: row['Ajia-3_v'] if row['Ajia-5_v'] == 0 and row['Ajia-3_v'] > 0 else row['Ajia-5_v'],
                axis=1
            )
            print('----------------LLM预测的列表---------------------')
            print(str(list(events_2['new_column'])))

            try:
                a, b, c = get_result(str(list(events_2['new_column'])), 1)
            except Exception as e:
                print(f"An error occurred: {e}")
                a, b, c = -100, -100, -100
            print('----------------预测的值---------------------')
            print(a, b, c)

            indices = events_2.index[events_2['new_column'] == a].tolist()
            df.loc[indices, 'status'] = 'A架摆出'

            indices = events_2.index[events_2['new_column'] == b].tolist()
            df.loc[indices, 'status'] = '征服者出水'
            previous_indices = [idx - 1 for idx in indices if idx > 0]
            df.loc[previous_indices, 'status'] = '缆绳挂妥'

            indices = events_2.index[events_2['new_column'] == c].tolist()
            df.loc[indices, 'status'] = '征服者落座'

        print('------------------')
        print(L4)
df = df.drop(columns=['date'])  # 删除 'date' 列
df = df.drop(columns=['check_current_presence'])  # 删除 'date' 列
#把这两个注释的原因是因为ai_brain中的api工具会用到check_current_presence这个字段，后悔了注释完效果更不好，当然也可能不是注释导致的问题
df.to_csv('database_in_use/Ajia_plc_1.csv', index=False)


开机时间: 2024-05-16 23:48:50, 关机时间: 2024-05-16 23:49:50
事件数量: 2
-----------------事件--------------------
[0.0, -1.0]
-----------------事件--------------------


C:\Users\dell\AppData\Local\Temp\ipykernel_2612\1784406115.py:576: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  events_2.loc[:, 'csvTime'] = pd.to_datetime(events_2['csvTime'])


------------------
[]
开机时间: 2024-05-17 10:14:50, 关机时间: 2024-05-17 10:15:50
事件数量: 2
-----------------事件--------------------
[0.0, -1.0]
-----------------事件--------------------


C:\Users\dell\AppData\Local\Temp\ipykernel_2612\1784406115.py:576: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  events_2.loc[:, 'csvTime'] = pd.to_datetime(events_2['csvTime'])


------------------
[]
开机时间: 2024-05-17 19:22:50, 关机时间: 2024-05-17 19:23:50
事件数量: 2
-----------------事件--------------------
[0.0, -1.0]
-----------------事件--------------------


...

------------------
[]
开机时间: 2024-09-14 11:37:55, 关机时间: 2024-09-14 11:38:55
事件数量: 2
-----------------事件--------------------
[0.0, -1.0]
-----------------事件--------------------


C:\Users\dell\AppData\Local\Temp\ipykernel_2612\1784406115.py:576: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  events_2.loc[:, 'csvTime'] = pd.to_datetime(events_2['csvTime'])


------------------
[]
开机时间: 2024-09-14 15:23:55, 关机时间: 2024-09-14 15:24:55
事件数量: 2
-----------------事件--------------------
[0.0, -1.0]
-----------------事件--------------------


C:\Users\dell\AppData\Local\Temp\ipykernel_2612\1784406115.py:576: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  events_2.loc[:, 'csvTime'] = pd.to_datetime(events_2['csvTime'])


------------------
[]


In [12]:

# In[4]:


import pandas as pd

# 读取CSV文件
df = pd.read_csv('data/Port3_ksbg_9.csv')
# 将P3_33列转换为数值类型，无法转换的保留原值
df['P3_33'] = pd.to_numeric(df['P3_33'], errors='coerce')
# 初始化status列
df['status'] = 'False'
# A架开机关机
for i in range(1, df.shape[0]):
    # 开机  前一个值为0
  #当前值大于0
    if df.loc[i - 1, 'P3_33'] == 0 and df.loc[i, 'P3_33'] > 0:
        df.loc[i, 'status'] = 'ON_DP'
    # 关机
    if df.loc[i - 1, 'P3_33'] > 0 and df.loc[i, 'P3_33'] == 0:
        df.loc[i, 'status'] = 'OFF_DP'
# 保存结果
df.to_csv('database_in_use/Port3_ksbg_9.csv', index=False)

In [ ]:

# In[5]:


# 读取CSV文件
df = pd.read_csv('data/device_13_11_meter_1311.csv')

# 将13-11-6_v列转换为数值类型，无法转换的保留原值
df['13-11-6_v'] = pd.to_numeric(df['13-11-6_v'], errors='coerce')

# 初始化status和action列
df['status'] = 'False'
df['action'] = 'False'


def sliding_window_5(arr):
    """
    使用大小为5的滑动窗口处理数据序列
    
    功能说明：
    检测并修正数据中的异常下降-上升模式
    当发现中间三个值异常降低（<10）而两端正常（>10）时，
    用第一个正常值替换这三个异常值
    
    参数：
    arr (list): 输入的数值列表，通常是电流值序列
    
    返回：
    list: 处理后的新数组，保持原数组长度不变
    """
    # 设置滑动窗口大小为5
    window_size = 5
    
    # 创建输入数组的深拷贝，避免修改原始数据
    modified_arr = arr.copy()
    
    # 遍历所有可能的窗口位置
    # 循环范围：从0到len(arr)-window_size+1，确保窗口不会超出数组边界
    for i in range(len(arr) - window_size + 1):
        # 提取当前窗口的5个值
        window = arr[i:i + window_size]
        
        # 判断当前窗口是否满足特定模式：
        # window[0]: 第一个值 (>10，正常值)
        # window[1:4]: 中间三个值 (<10，异常低值)
        # window[4]: 最后一个值 (>10，正常值)
        if (window[1] < 10 and           # 第二个值小于10
            window[2] < 10 and           # 第三个值小于10
            window[3] < 10 and           # 第四个值小于10
            window[0] > 10 and           # 第一个值大于10
            window[4] > 10):             # 最后一个值大于10
            
            # 使用第一个正常值(window[0])替换中间三个异常值
            # modified_arr[i + 1:i + 4]选择要替换的三个位置
            # [window[0]] * 3 创建一个包含三个相同值的列表
            modified_arr[i + 1:i + 4] = [window[0]] * 3
            
    return modified_arr


def sliding_window_4(arr):
    """滑动窗口大小为4的逻辑"""
    window_size = 4
    modified_arr = arr.copy()
    for i in range(len(arr) - window_size + 1):
        window = arr[i:i + window_size]
        if window[1] < 10 and window[2] < 10 and window[0] > 10 and window[3] > 10:
            # 将 window[0] 包装成列表进行赋值
            modified_arr[i + 1:i + 3] = [window[0]] * 2
    return modified_arr


def sliding_window_3(arr):
    """滑动窗口大小为3的逻辑"""
    window_size = 3
    modified_arr = arr.copy()
    for i in range(len(arr) - window_size + 1):
        window = arr[i:i + window_size]
        if window[1] < 10 and window[0] > 10 and window[2] > 10:
            # 直接赋值，因为只修改一个值
            modified_arr[i + 1] = window[0]
    return modified_arr


# 应用滑动窗口逻辑到 DataFrame 的某一列
df['13-11-6_v_new'] = sliding_window_5(df['13-11-6_v'].tolist())
df['13-11-6_v_new'] = sliding_window_4(df['13-11-6_v_new'].tolist())
df['13-11-6_v_new'] = sliding_window_3(df['13-11-6_v_new'].tolist())

# 检测折臂吊车的开机和关机事件
segments = []
start_time = None

for i in range(1, df.shape[0]):
    # 开机
    if df.iloc[i - 1]['13-11-6_v'] == 0 and df.iloc[i]['13-11-6_v'] > 0:
        df.at[df.index[i], 'status'] = '折臂吊车开机'
    # 关机
    if df.iloc[i - 1]['13-11-6_v'] > 0 and df.iloc[i]['13-11-6_v'] == 0:
        df.at[df.index[i], 'status'] = '折臂吊车关机'

    # 检测由待机进入工作和由工作进入待机的事件
    if df.iloc[i - 1]['13-11-6_v_new'] < 10 and df.iloc[i]['13-11-6_v_new'] > 10:
        df.at[df.index[i], 'action'] = '由待机进入工作'
    if df.iloc[i - 1]['13-11-6_v_new'] > 10 and df.iloc[i]['13-11-6_v_new'] < 10:
        df.at[df.index[i], 'action'] = '由工作进入待机'
    # 遍历DataFrame
for index, row in df.iterrows():
    if row['status'] == '折臂吊车开机':
        start_time = row['csvTime']
    elif row['status'] == '折臂吊车关机' and start_time is not None:
        end_time = row['csvTime']
        segments.append((start_time, end_time))
        start_time = None
        
        
from collections import Counter


def find_most_frequent_number(lst):
    # 使用 Counter 统计每个数的出现次数
    counter = Counter(lst)
    # 找到出现次数最多的数（如果有多个，只返回第一个）
    most_common_number = counter.most_common(1)[0][0]
    return most_common_number


# 提取每个区段内的“由待机进入工作”和“由工作进入待机”事件
for segment in segments:
    start, end = segment
    events = df[
        (df['csvTime'] >= start) & (df['csvTime'] <= end) & (df['action'].isin(['由待机进入工作', '由工作进入待机']))]
    events_2 = df[(df['csvTime'] >= start) & (df['csvTime'] <= end)]
    # 检查事件数量是否为偶数且等于6
    if events.shape[0] == 6:
        print(f"开机时间: {start}, 关机时间: {end}")
        print(f"事件数量: {events.shape[0]}")
        # 处理每一对事件
        for i in range(0, 6, 2):

            event_start = events.iloc[i]
            event_end = events.iloc[i + 1]

            if event_start['action'] == '由待机进入工作' and event_end['action'] == '由工作进入待机':
                start_event_time = event_start['csvTime']
                end_event_time = event_end['csvTime']
                between_events = df[(df['csvTime'] >= start_event_time) & (df['csvTime'] <= end_event_time)]
                data1 = list(between_events['13-11-6_v'])

                # 找到最后一个大于9的值
                last_value_above_9 = next((x for x in reversed(data1) if x > 9), None)

                if last_value_above_9 is not None:
                    all_indices = between_events.index[between_events['13-11-6_v_new'] == last_value_above_9].tolist()
                    last_index = all_indices[-1] if all_indices else None

                    # 根据事件对的顺序更新status
                    if last_index is not None:
                        if i == 0:
                            df.loc[last_index, 'status'] = '小艇检查完毕'
                        elif i == 2:
                            df.loc[last_index, 'status'] = '小艇入水'
                        elif i == 4:
                            df.loc[last_index, 'status'] = '小艇落座'
                else:
                    print("列表中没有大于 9 的值")
    if events.shape[0] == 4:
        print(f"开机时间: {start}, 关机时间: {end}")
        print(f"事件数量: {events.shape[0]}")
        # 处理每一对事件
        for i in range(0, 4, 2):
            event_start = events.iloc[i]
            event_end = events.iloc[i + 1]
            if event_start['action'] == '由待机进入工作' and event_end['action'] == '由工作进入待机':
                start_event_time = event_start['csvTime']
                end_event_time = event_end['csvTime']
                between_events = df[(df['csvTime'] >= start_event_time) & (df['csvTime'] <= end_event_time)]
                data1 = list(between_events['13-11-6_v'])

                # 找到最后一个大于9的值
                last_value_above_9 = next((x for x in reversed(data1) if x > 9), None)

                if last_value_above_9 is not None:
                    all_indices = between_events.index[between_events['13-11-6_v_new'] == last_value_above_9].tolist()
                    last_index = all_indices[-1] if all_indices else None

                    # 根据事件对的顺序更新status

                    if last_index is not None and df.loc[last_index, 'status'] == "FALSE":
                        if i == 0:
                            df.loc[last_index, 'status'] = '小艇入水'
                        elif i == 2:
                            df.loc[last_index, 'status'] = '小艇落座'
                else:
                    print("列表中没有大于 9 的值")
                # 保存结果
df = df.drop(columns=['action'])
df = df.drop(columns=['13-11-6_v_new'])

df.to_csv('database_in_use/device_13_11_meter_1311.csv', index=False)

In [13]:


# In[6]:


# Pre3: Data Annotations
def create_annotations():
    df_desc = pd.read_csv(f'{data_path}字段释义.csv', encoding='gbk')
    df_desc['字段含义_new'] = df_desc['字段含义'] + df_desc['单位'].apply(
        lambda x: f",单位:{x}" if pd.notnull(x) else "")
    field_dict = df_desc.set_index('字段名')['字段含义_new'].to_dict()

    folder_path = 'database_in_use'
    files = [f.split('.')[0] for f in os.listdir(folder_path) if f.endswith('.csv')]

    descriptions = []
    for file_name in files:
        df = pd.read_csv(f'{folder_path}/{file_name}.csv')
        columns = df.columns.tolist()
        annotations = [field_dict.get(col, '无注释') for col in columns]
        descriptions.append({'数据表名': file_name, '字段名': columns, "字段含义": annotations})

    # Customize specific tables
    for item in descriptions:
        if item['数据表名'] == 'Ajia_plc_1':
            item['字段含义'][-2] = 'A架动作,包括关机、开机、A架摆出、缆绳挂妥、征服者出水、征服者落座、征服者起吊、征服者入水、缆绳解除、A架摆回'
        if item['数据表名'] == 'device_13_11_meter_1311':
            item['字段含义'][-1] = '折臂吊车及小艇动作,包括折臂吊车关机,折臂吊车开机,小艇检查完毕,小艇入水,小艇落座'
        if item['数据表名'] == 'Port3_ksbg_9':
            item['字段含义'][-1] = 'DP动作,包括OFF_DP,ON_DP'

    with open('dict.json', 'w', encoding='utf-8') as f:
        json.dump(descriptions, f, ensure_ascii=False, indent=4)


# create_annotations()
# In[7]:
import pandas as pd

df = pd.read_csv(f'{data_path}字段释义.csv', encoding='gbk')
# 检查某一列是否有重复值
column_name = '字段名'
value_counts = df[column_name].value_counts()
if any(value_counts > 1):
    print(f"列 '{column_name}' 中存在重复值。")
else:
    print(f"列 '{column_name}' 中没有重复值。")
df['字段含义_new'] = df['字段含义'] + df['单位'].apply(lambda x: ",单位:" + x if pd.notnull(x) else "")
# 将两列转换为字典
field_dict = df.set_index('字段名')['字段含义_new'].to_dict()


# 读取CSV文件

def aa(filename):
    df = pd.read_csv(f'database_in_use/{filename}.csv')
    if 'Unnamed: 0' in df.columns:
        del df['Unnamed: 0']
    # 获取列名
    column_names = df.columns.tolist()
    # 定义列名与中文注释的映射字典
    column_name_to_chinese = field_dict
    # 获取中文注释
    chinese_annotations = [column_name_to_chinese.get(col, '无注释') for col in column_names]

    last_dict = {'数据表名': filename, '字段名': column_names, "字段含义": chinese_annotations}
    return last_dict


def process_folder(folder_path):
    # 获取文件夹中的所有CSV文件
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    # 初始化结果列表
    result_list = []

    # 遍历每个CSV文件
    for csv_file in csv_files:
        # 去掉文件扩展名，获取文件名
        filename = os.path.splitext(csv_file)[0]
        # 调用aa函数处理文件
        result_dict = aa(filename)
        # 将结果字典添加到列表中
        result_list.append(result_dict)

    return result_list


#使用示例
folder_path = 'database_in_use'
os.makedirs(folder_path, exist_ok=True)
result = process_folder(folder_path)
result = [item for item in result if item['数据表名'] != '设备参数详情表']
for item in result:
    if item['数据表名'] == 'Ajia_plc_1':
        item['字段含义'][-1] = 'A架动作,包括关机、开机、A架摆出、缆绳挂妥、征服者出水、征服者落座、征服者起吊、征服者入水、缆绳解除、A架摆回'
    if item['数据表名'] == 'device_13_11_meter_1311':
        item['字段含义'][-1] = '折臂吊车及小艇动作,包括折臂吊车关机,折臂吊车开机,小艇检查完毕,小艇入水,小艇落座'
    if item['数据表名'] == 'Port3_ksbg_9':
        item['字段含义'][-1] = 'DP动作,包括OFF_DP,ON_DP'
# %%
df1 = pd.read_excel(f'{data_path}设备参数详情.xlsx', sheet_name='字段释义')
df1['含义1'] = df1['含义'].fillna('') + ',' + df1['备注'].fillna('')
dict_shebei = {'数据表名': '设备参数详情表', '字段名': list(df1['字段']), "字段含义": list(df1['含义1'])}
# 修改字段含义列表的第二个值
dict_shebei['字段含义'][1] = "参数中文名,值包含一号柴油发电机组滑油压力、停泊/应急发电机组、一号柴油发电机组滑油压力等"
result.append(dict_shebei)

# 假设这是你的列表数据
data_list = result
# 将列表存入 JSON 文件
with open('dict.json', 'w', encoding='utf-8') as f:
    json.dump(data_list, f, ensure_ascii=False, indent=4)



开机时间: 2024-05-16 22:14:41, 关机时间: 2024-05-16 23:40:06
事件数量: 6
开机时间: 2024-05-18 08:00:34, 关机时间: 2024-05-18 08:50:01
事件数量: 6
开机时间: 2024-05-18 16:07:42, 关机时间: 2024-05-18 16:56:51
事件数量: 6
开机时间: 2024-05-19 07:36:16, 关机时间: 2024-05-19 08:36:46
事件数量: 6
开机时间: 2024-05-19 16:02:50, 关机时间: 2024-05-19 17:16:13
事件数量: 4
开机时间: 2024-05-20 07:00:07, 关机时间: 2024-05-20 07:29:21
事件数量: 4
开机时间: 2024-05-21 11:41:40, 关机时间: 2024-05-21 12:12:42
事件数量: 4
开机时间: 2024-05-21 16:16:06, 关机时间: 2024-05-21 16:36:08
事件数量: 4
开机时间: 2024-05-25 08:00:31, 关机时间: 2024-05-25 08:58:52
事件数量: 6
开机时间: 2024-05-26 09:00:42, 关机时间: 2024-05-26 09:32:53
事件数量: 6
开机时间: 2024-05-26 16:04:42, 关机时间: 2024-05-26 17:14:57
事件数量: 6
开机时间: 2024-05-27 08:03:57, 关机时间: 2024-05-27 08:41:09
事件数量: 6
开机时间: 2024-05-27 09:00:14, 关机时间: 2024-05-27 09:29:28
事件数量: 4
开机时间: 2024-05-28 08:00:54, 关机时间: 2024-05-28 08:55:17
事件数量: 6
开机时间: 2024-05-29 18:00:52, 关机时间: 2024-05-30 10:00:31
事件数量: 4
开机时间: 2024-05-30 16:00:49, 关机时间: 2024-05-30 17:22:14
事件数量: 4
开机时间: 2024-08-15 08:06:1

In [14]:
# merge_csv_files(data_path,ouput_path)